In [1]:
import rasterio
import rasterio.warp
from rasterio.enums import Resampling
import geopandas as gpd
import pandas as pd
import numpy as np
import os
from shapely.geometry import box
print("Libraries imported successfully!")

Libraries imported successfully!


In [3]:
# --- Define Master Grid Parameters --- These parameters will determine the common coordinate system, resolution and spatial extent for all input data.

try:
    if 'CONDA_PREFIX' in os.environ:
        conda_prefix = os.environ['CONDA_PREFIX']
        proj_lib_path = os.path.join(conda_prefix, 'Library', 'share', 'proj') # PROJ data files
        
        if os.path.exists(gdal_data_path):
            os.environ['GDAL_DATA'] = gdal_data_path
            print(f"GDAL_DATA environment variable set to: {os.environ['GDAL_DATA']}")
        else:
            print(f"Warning: GDAL_DATA path not found at {gdal_data_path}. GDAL_DATA may still be unset.")

        if os.path.exists(proj_lib_path):
            os.environ['PROJ_LIB'] = proj_lib_path
            print(f"PROJ_LIB environment variable set to: {os.environ['PROJ_LIB']}")
        else:
            print(f"Warning: PROJ_LIB path not found at {proj_lib_path}. PROJ_LIB may still be unset.")
    else:
        print("Warning: CONDA_PREFIX not found in environment variables. GDAL_DATA and PROJ_LIB may not be set automatically.")
except Exception as e:
    print(f"Error setting GDAL_DATA/PROJ_LIB: {e}")

# 1. Define the Target Coordinate Reference System (CRS)
target_crs = 'EPSG:32643'                                                                          # User input
print(f"Target CRS set to: {target_crs}")

# 2. Define the Target Resolution (Cell Size)
target_resolution = 100                                                                            # User input
print(f"Target Resolution set to: {target_resolution} meters")

# 3. Define the Spatial Extent of the Master Grid
shapefile_path = r'E:\Hackathon\GIS\block.shp'                                                     # User input
print(f"Attempting to read shapefile from: {shapefile_path}")

try:
    gdf = gpd.read_file(shapefile_path)
    gdf = gdf.to_crs(target_crs)
    target_bounds = gdf.total_bounds  
    print(f"Master Grid Bounds derived from shapefile: {target_bounds}")

    dst_width = int((target_bounds[2] - target_bounds[0]) / target_resolution)
    dst_height = int((target_bounds[3] - target_bounds[1]) / target_resolution)

    master_transform = rasterio.transform.from_bounds(*target_bounds,
                                                     width=dst_width,
                                                     height=dst_height)
    print(f"Master Grid Dimensions (pixels): Width={dst_width}, Height={dst_height}")
    print(f"Master Grid Transform created.")

except FileNotFoundError:
    print(f"Error: Shapefile not found at {shapefile_path}. Please check the path.")
except Exception as e:
    print(f"An error occurred while processing the shapefile: {e}")
    target_bounds = None 
    dst_width = 0
    dst_height = 0
    master_transform = None

if target_bounds is not None:
    master_grid_params = {
        'target_crs': target_crs,
        'target_resolution': target_resolution,
        'target_bounds': target_bounds,
        'dst_width': dst_width,
        'dst_height': dst_height,
        'master_transform': master_transform
    }
    print("\nMaster grid parameters initialized successfully!")
    print("You can now proceed to the next cell to define the raster processing function.")
else:
    master_grid_params = None # Indicate that initialization failed
    print("\nMaster grid parameters could not be fully initialized due to errors above.")
    print("Please resolve the errors (especially the shapefile path) before proceeding.")

Error setting GDAL_DATA/PROJ_LIB: name 'gdal_data_path' is not defined
Target CRS set to: EPSG:32643
Target Resolution set to: 100 meters
Attempting to read shapefile from: E:\Hackathon\GIS\block.shp
Master Grid Bounds derived from shapefile: [ 607128.26186922 1520302.09484182  796751.60625961 1743202.77801102]
Master Grid Dimensions (pixels): Width=1896, Height=2229
Master Grid Transform created.

Master grid parameters initialized successfully!
You can now proceed to the next cell to define the raster processing function.


In [4]:
# --- Define the process_raster function
def process_raster(input_raster_path, master_grid_params, resampling_method=Resampling.bilinear, is_target_layer=False):
    target_crs = master_grid_params['target_crs'] 
    target_transform = master_grid_params['master_transform'] 
    dst_width = master_grid_params['dst_width'] 
    dst_height = master_grid_params['dst_height'] 

    if is_target_layer:
        chosen_resampling = Resampling.nearest 
        print(f"  Processing TARGET layer '{os.path.basename(input_raster_path)}' with NEAREST NEIGHBOR resampling.") 
    else:
        chosen_resampling = resampling_method 
        print(f"  Processing FEATURE layer '{os.path.basename(input_raster_path)}' with {chosen_resampling.name} resampling.") 

    try:
        with rasterio.open(input_raster_path) as src: 
            destination = np.zeros((dst_height, dst_width), dtype=src.profile['dtype']) 

            reproject_profile = src.profile.copy() 
            reproject_profile.update({
                'crs': target_crs,
                'transform': target_transform,
                'width': dst_width,
                'height': dst_height,
                'driver': 'GTiff',
                'dtype': src.profile['dtype']
            }) #

            rasterio.warp.reproject(
                source=rasterio.band(src, 1), 
                destination=destination, 
                src_transform=src.transform, 
                src_crs=src.crs, 
                dst_transform=target_transform, 
                dst_crs=target_crs, 
                resampling=chosen_resampling, 
                num_threads=os.cpu_count() 
            )
            return destination.flatten() 

    except Exception as e:
        print(f"Error processing {input_raster_path}: {e}") 
        return None

# --- List Your Input Raster Files ---
target_layer_path = r'E:\\Hackathon\\Target\\FeMn_target.tif'                                   # User input

feature_raster_files = [                                                                        # User input
    r'E:\\Hackathon\\Geology\\norm_Lith_FeMn.tif',
    r'E:\\Hackathon\\Geology\\norm_gr_acfm.tif',
    r'E:\\Hackathon\\Geology\\norm_Line_density.tif',
    r'E:\\Hackathon\\Geology\\norm_Line_ring.tif',
    r'E:\\Hackathon\\Geology\\norm_intersect.tif',
    r'E:\\Hackathon\\Geology\\norm_gr_sch.tif',
    r'E:\\Hackathon\\Geology\\norm_vein.tif',
    r'E:\\Hackathon\\Geology\\norm_dyke.tif',
    r'E:\\Hackathon\\NGCM\\norm_pca1.tif',
    r'E:\\Hackathon\\Geophy\\normalized_BA.tif',
    r'E:\\Hackathon\\Geophy\\normalized_BA_res.tif',
    r'E:\\Hackathon\\Geophy\\normalized_BA_res_vdr.tif',
    r'E:\\Hackathon\\Geophy\\normalized_BA_tdr.tif',
    r'E:\\Hackathon\\Geophy\\normalized_BA_up5.tif',
    r'E:\\Hackathon\\Geophy\\normalized_MA.tif',
    r'E:\\Hackathon\\Geophy\\norm_MA_rtp.tif',
    r'E:\\Hackathon\\Geophy\\norm_MA_rtpres.tif',
    r'E:\\Hackathon\\Geophy\\normalized_MA_ana.tif',
    r'E:\\Hackathon\\Geophy\\normalized_MA_res.tif',
    r'E:\\Hackathon\\Geophy\\normalized_MA_up5.tif',
    r'E:\\Hackathon\\PGRS\\normalized_chl.tif',
    r'E:\\Hackathon\\PGRS\\norm_Silica.tif',
    r'E:\\Hackathon\\PGRS\\normalized_feo.tif',
    r'E:\\Hackathon\\PGRS\\normalized_mf.tif',
]

print("Feature raster files to process:") 
for f in feature_raster_files:
    print(f" - {f}") 
print(f"\nTarget layer: {target_layer_path}") 

all_flattened_data = [] 
target_flattened_data = None 

if 'master_grid_params' not in locals() or master_grid_params is None:
    print("\nERROR: Master grid parameters were not initialized. Please go back to Step 1 and resolve the issues.") 
else:
    print("\nStarting FEATURE raster processing...") 
    for i, r_file in enumerate(feature_raster_files):
        processed_array = process_raster(r_file, master_grid_params, resampling_method=Resampling.bilinear, is_target_layer=False) 
        if processed_array is not None:
            all_flattened_data.append(processed_array) 
        else:
            print(f"Skipping {r_file} due to processing error.") 

    print("\nStarting TARGET layer processing...") 
    target_flattened_data = process_raster(target_layer_path, master_grid_params, is_target_layer=True) 
    if target_flattened_data is not None:
        print(f"Target layer '{os.path.basename(target_layer_path)}' processed successfully.") 
    else:
        print(f"ERROR: Target layer '{os.path.basename(target_layer_path)}' could not be processed.") 

    print("\nAll rasters attempted to be processed.") 

    expected_length = master_grid_params['dst_width'] * master_grid_params['dst_height'] 

    if not all_flattened_data and target_flattened_data is None:
        print("ERROR: No raster data (features or target) was successfully processed. Cannot create DataFrame.") 
    elif target_flattened_data is None:
        print("ERROR: Target layer was not processed successfully. Cannot create DataFrame.") 
    elif len(all_flattened_data) > 0 and len(set(len(arr) for arr in all_flattened_data)) != 1:
        print("\nERROR: Feature data arrays have inconsistent lengths!") #
        print("Expected length (from master grid dimensions):", expected_length) 
        for i, arr in enumerate(all_flattened_data):
            print(f"  Array from {feature_raster_files[i]}: Length = {len(arr)}") 
        print("Please review your master grid parameters and feature raster files. Cannot proceed to CSV creation.") 
    elif len(target_flattened_data) != expected_length:
        print(f"\nERROR: Target layer array has inconsistent length! Expected {expected_length}, got {len(target_flattened_data)}.") 
        print("Please review your master grid parameters and target raster file. Cannot proceed to CSV creation.") 
    elif len(all_flattened_data) > 0 and len(all_flattened_data[0]) != expected_length:
         print(f"\nERROR: First feature array length ({len(all_flattened_data[0])}) does not match expected length ({expected_length}).") 
         print("Please review your master grid parameters and feature raster files. Cannot proceed to CSV creation.") 
    else:
        print(f"\nAll processed rasters (features and target) have consistent lengths: {expected_length}") 
        print("Ready to combine data into a DataFrame and save to CSV.") 

        processed_data_arrays = all_flattened_data + [target_flattened_data] 
        # FIX: Add the following line to define 'raster_files' for the next cell
        raster_files = feature_raster_files + [target_layer_path] 

        print("\nProcessed data stored. You can now proceed to the next cell to create the final CSV.") 

Feature raster files to process:
 - E:\\Hackathon\\Geology\\norm_Lith_FeMn.tif
 - E:\\Hackathon\\Geology\\norm_gr_acfm.tif
 - E:\\Hackathon\\Geology\\norm_Line_density.tif
 - E:\\Hackathon\\Geology\\norm_Line_ring.tif
 - E:\\Hackathon\\Geology\\norm_intersect.tif
 - E:\\Hackathon\\Geology\\norm_gr_sch.tif
 - E:\\Hackathon\\Geology\\norm_vein.tif
 - E:\\Hackathon\\Geology\\norm_dyke.tif
 - E:\\Hackathon\\NGCM\\norm_pca1.tif
 - E:\\Hackathon\\Geophy\\normalized_BA.tif
 - E:\\Hackathon\\Geophy\\normalized_BA_res.tif
 - E:\\Hackathon\\Geophy\\normalized_BA_res_vdr.tif
 - E:\\Hackathon\\Geophy\\normalized_BA_tdr.tif
 - E:\\Hackathon\\Geophy\\normalized_BA_up5.tif
 - E:\\Hackathon\\Geophy\\normalized_MA.tif
 - E:\\Hackathon\\Geophy\\norm_MA_rtp.tif
 - E:\\Hackathon\\Geophy\\norm_MA_rtpres.tif
 - E:\\Hackathon\\Geophy\\normalized_MA_ana.tif
 - E:\\Hackathon\\Geophy\\normalized_MA_res.tif
 - E:\\Hackathon\\Geophy\\normalized_MA_up5.tif
 - E:\\Hackathon\\PGRS\\normalized_chl.tif
 - E:\\Hackatho

In [5]:
output_directory = r'E:\\Hackathon\\Model\\FeMn'                                                  # User input
output_csv_filename = 'mineral_prospectivity_FeMn.csv'                                            # User input

os.makedirs(output_directory, exist_ok=True) 
print(f"Ensured output directory exists: {output_directory}") 

output_csv_path = os.path.join(output_directory, output_csv_filename) 
print(f"Output CSV will be saved to: {output_csv_path}") 

if 'processed_data_arrays' not in locals() or not processed_data_arrays:
    print("ERROR: 'processed_data_arrays' not found or is empty. Please ensure Step 3 ran successfully.") 
elif 'master_grid_params' not in locals() or master_grid_params is None:
    print("ERROR: 'master_grid_params' not found or is None. Please ensure Step 1 ran successfully.") 
elif 'raster_files' not in locals():
    print("ERROR: 'raster_files' list (from Step 3) not found. Cannot automatically name columns.") 
    print("Please ensure Step 3 ran successfully and 'raster_files' was defined.") #
else:
    print("\nProceeding to create final DataFrame...") 

    # 2. Add X, Y Coordinates to the DataFrame FIRST
    target_bounds = master_grid_params['target_bounds'] 
    target_resolution = master_grid_params['target_resolution'] 
    dst_width = master_grid_params['dst_width'] 
    dst_height = master_grid_params['dst_height'] 

    x_coords = np.linspace(target_bounds[0] + target_resolution / 2,
                           target_bounds[2] - target_resolution / 2,
                           dst_width) 

    y_coords = np.linspace(target_bounds[3] - target_resolution / 2,
                           target_bounds[1] + target_resolution / 2,
                           dst_height) 

    xv, yv = np.meshgrid(x_coords, y_coords) 

    # Create a DataFrame directly from the coordinates
    df = pd.DataFrame({
        'X_Coordinate': xv.flatten(),
        'Y_Coordinate': yv.flatten()
    }) 
    print("X, Y coordinates DataFrame initialized.") 

    # 3. Add predictor factors to the DataFrame
    column_names_factors = [os.path.splitext(os.path.basename(r_file))[0] for r_file in raster_files] #
    print(f"Adding predictor factors with names: {column_names_factors}") 
    df_factors = pd.DataFrame(np.array(processed_data_arrays).T,
                              columns=column_names_factors) 
    df = pd.concat([df, df_factors], axis=1) 

    print(f"Final DataFrame created with {len(df)} rows and {df.shape[1]} columns.") 
    print(f"Column order: {df.columns.tolist()}") 


    # 4. Handle NoData Values
    initial_rows = len(df) 
    df.replace([np.inf, -np.inf], np.nan, inplace=True) 
    df_factor_columns = column_names_factors #
    df.dropna(subset=df_factor_columns, inplace=True) 

    print(f"Handled NoData values: Dropped {initial_rows - len(df)} rows containing NaN from factor columns.") 

    # 5. Save the DataFrame to the specified CSV file path
    df.to_csv(output_csv_path, index=False) 

    print(f"\nFinal DataFrame successfully saved to: {output_csv_path}") 
    print(f"The CSV contains {len(df)} rows and {df.shape[1]} columns.") 

Ensured output directory exists: E:\\Hackathon\\Model\\FeMn
Output CSV will be saved to: E:\\Hackathon\\Model\\FeMn\mineral_prospectivity_FeMn.csv

Proceeding to create final DataFrame...
X, Y coordinates DataFrame initialized.
Adding predictor factors with names: ['norm_Lith_FeMn', 'norm_gr_acfm', 'norm_Line_density', 'norm_Line_ring', 'norm_intersect', 'norm_gr_sch', 'norm_vein', 'norm_dyke', 'norm_pca1', 'normalized_BA', 'normalized_BA_res', 'normalized_BA_res_vdr', 'normalized_BA_tdr', 'normalized_BA_up5', 'normalized_MA', 'norm_MA_rtp', 'norm_MA_rtpres', 'normalized_MA_ana', 'normalized_MA_res', 'normalized_MA_up5', 'normalized_chl', 'norm_Silica', 'normalized_feo', 'normalized_mf', 'FeMn_target']
Final DataFrame created with 4226184 rows and 27 columns.
Column order: ['X_Coordinate', 'Y_Coordinate', 'norm_Lith_FeMn', 'norm_gr_acfm', 'norm_Line_density', 'norm_Line_ring', 'norm_intersect', 'norm_gr_sch', 'norm_vein', 'norm_dyke', 'norm_pca1', 'normalized_BA', 'normalized_BA_res', 